# Bài 7: Xây dựng Chatbot RAG với LangFuse

## Mục tiêu học tập
- Hiểu cách thiết kế và triển khai pipeline chatbot Q&A dựa trên RAG
- Sử dụng LangFuse để theo dõi và tối ưu hóa hiệu suất chatbot
- Triển khai đánh giá tự động và thu thập feedback từ người dùng
- Phân tích metrics để cải thiện chất lượng chatbot

## Giới thiệu

Chatbot RAG (Retrieval-Augmented Generation) là một trong những ứng dụng phổ biến nhất của AI trong thực tế. Tuy nhiên, việc xây dựng một chatbot RAG hiệu quả gặp phải nhiều thách thức:

### 🚧 Thách thức chính:
- **Hallucination**: Model tạo ra thông tin sai lệch
- **Độ liên quan**: Truy xuất tài liệu không phù hợp với câu hỏi
- **Chất lượng câu trả lời**: Câu trả lời mơ hồ hoặc không đầy đủ
- **Hiệu suất**: Độ trễ cao và sử dụng token không tối ưu
- **Trải nghiệm người dùng**: Thiếu context và khả năng xử lý multi-turn conversation

### 💡 Giải pháp với LangFuse:
- **Observability**: Theo dõi toàn bộ pipeline RAG
- **Evaluation**: Đánh giá tự động chất lượng câu trả lời
- **Optimization**: Tối ưu hóa dựa trên dữ liệu thực tế
- **User Feedback**: Thu thập và phân tích feedback người dùng

## Cài đặt & Cấu hình

In [ ]:
import os
import uuid
from datetime import datetime
from typing import List, Dict, Optional, Tuple

# LangChain imports
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# LangFuse
from langfuse import Langfuse
from langfuse.decorators import observe, langfuse_context
from langfuse.callback import CallbackHandler

# Other imports
import json
import time
from dataclasses import dataclass

In [ ]:
# Cấu hình environment variables
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..."
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # hoặc self-hosted URL
os.environ["ANTHROPIC_API_KEY"] = "sk-ant-..."

# Khởi tạo LangFuse client
langfuse = Langfuse()
langfuse_handler = CallbackHandler()

print("✅ LangFuse đã được cấu hình thành công!")

## Use Case: Chatbot Hỗ trợ Khách hàng Sản phẩm

Chúng ta sẽ xây dựng một chatbot để trả lời câu hỏi về các sản phẩm của một công ty công nghệ.

In [ ]:
# Dữ liệu sản phẩm mẫu
PRODUCT_DOCUMENTS = [
    {
        "content": """MacBook Pro 14-inch với chip M3 Pro là laptop cao cấp dành cho các chuyên gia sáng tạo. 
        Sản phẩm có màn hình Liquid Retina XDR 14.2 inch, thời lượng pin lên đến 18 giờ, 
        và hỗ trợ kết nối Thunderbolt 4. Giá từ 49.999.000 VNĐ. 
        Bảo hành 12 tháng chính hãng, hỗ trợ trả góp 0% lãi suất.""",
        "metadata": {"product": "MacBook Pro 14", "category": "laptop", "price": 49999000}
    },
    {
        "content": """iPhone 15 Pro Max là flagship smartphone với camera 48MP, 
        chip A17 Pro được sản xuất trên tiến trình 3nm. Màn hình Super Retina XDR 6.7 inch, 
        thân máy làm từ titanium cao cấp. Giá từ 32.999.000 VNĐ. 
        Có các màu: Natural Titanium, Blue Titanium, White Titanium, Black Titanium.""",
        "metadata": {"product": "iPhone 15 Pro Max", "category": "smartphone", "price": 32999000}
    },
    {
        "content": """AirPods Pro (2nd generation) với chip H2 mang lại chất lượng âm thanh vượt trội. 
        Tính năng chống ồn chủ động được cải thiện gấp đôi, thời lượng nghe nhạc 6 giờ, 
        case sạc không dây MagSafe. Giá 6.999.000 VNĐ. 
        Tương thích với tất cả thiết bị Apple, hỗ trợ Spatial Audio.""",
        "metadata": {"product": "AirPods Pro 2", "category": "audio", "price": 6999000}
    },
    {
        "content": """Chính sách bảo hành: Tất cả sản phẩm Apple chính hãng được bảo hành 12 tháng. 
        Khách hàng có thể mở rộng bảo hành lên 24 tháng với phí 10% giá trị sản phẩm. 
        Bảo hành bao gồm lỗi kỹ thuật, không bao gồm hư hỏng do tác động vật lý. 
        Trung tâm bảo hành tại Hà Nội, TP.HCM, Đà Nẵng.""",
        "metadata": {"type": "policy", "category": "warranty"}
    },
    {
        "content": """Chính sách vận chuyển: Miễn phí giao hàng toàn quốc cho đơn hàng từ 10 triệu. 
        Giao hàng nhanh trong ngày tại Hà Nội và TP.HCM với phí 50.000 VNĐ. 
        Giao hàng tiêu chuẩn 2-3 ngày làm việc. Hỗ trợ giao hàng đến tận nơi, 
        kiểm tra sản phẩm trước khi thanh toán.""",
        "metadata": {"type": "policy", "category": "shipping"}
    }
]

# Chuyển đổi thành Document objects
documents = [Document(page_content=doc["content"], metadata=doc["metadata"]) 
             for doc in PRODUCT_DOCUMENTS]

print(f"📄 Đã tạo {len(documents)} tài liệu sản phẩm")

## Xây dựng Vector Store và RAG Pipeline

In [ ]:
# Khởi tạo embeddings và vector store
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

# Tạo vector store từ documents
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="product_knowledge"
)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Lấy top 3 documents liên quan nhất
)

print("🔍 Vector store và retriever đã được thiết lập!")

In [ ]:
# Khởi tạo LLM
llm = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    temperature=0.1,
    callbacks=[langfuse_handler]
)

print("🤖 Claude đã sẵn sàng!")

## Thiết kế Chatbot RAG với Multi-turn Conversation

In [ ]:
@dataclass
class ChatMessage:
    """Lớp đại diện cho một tin nhắn trong cuộc hội thoại"""
    role: str  # 'user' hoặc 'assistant'
    content: str
    timestamp: datetime
    metadata: Optional[Dict] = None

class RAGChatbot:
    """Chatbot RAG với khả năng multi-turn conversation và LangFuse tracking"""
    
    def __init__(self, retriever, llm, langfuse_client):
        self.retriever = retriever
        self.llm = llm
        self.langfuse = langfuse_client
        self.conversation_history: List[ChatMessage] = []
        self.session_id = str(uuid.uuid4())
        
        # Thiết lập system prompt
        self.system_prompt = """Bạn là trợ lý AI chuyên nghiệp hỗ trợ khách hàng về các sản phẩm Apple.
        
Nhiệm vụ:
- Trả lời câu hỏi chính xác dựa trên thông tin được cung cấp
- Sử dụng ngôn ngữ thân thiện, chuyên nghiệp
- Nếu không có thông tin đủ, hãy thành thật nói không biết
- Đề xuất sản phẩm phù hợp nếu phù hợp
- Luôn cung cấp thông tin giá cả và chính sách nếu có

Quy tắc quan trọng:
- KHÔNG bịa đặt thông tin không có trong tài liệu
- Luôn trích dẫn thông tin từ nguồn được cung cấp
- Nếu khách hàng hỏi về sản phẩm không có, hãy đề xuất sản phẩm tương tự"""
    
    @observe(name="query_expansion")
    def expand_query(self, user_question: str) -> str:
        """Mở rộng và cải thiện câu hỏi của người dùng để tìm kiếm tốt hơn"""
        
        # Lấy context từ lịch sử hội thoại
        recent_context = ""
        if len(self.conversation_history) > 0:
            recent_msgs = self.conversation_history[-4:]  # Lấy 4 tin nhắn gần nhất
            recent_context = "\n".join([f"{msg.role}: {msg.content}" for msg in recent_msgs])
        
        expansion_prompt = ChatPromptTemplate.from_messages([
            ("system", """Bạn là chuyên gia phân tích câu hỏi. Nhiệm vụ của bạn là:
            1. Phân tích câu hỏi của người dùng trong context cuộc hội thoại
            2. Tạo ra câu hỏi tìm kiếm tối ưu để truy xuất thông tin liên quan
            3. Chỉ trả về câu hỏi đã được cải thiện, không giải thích
            
            Ví dụ:
            - "Giá bao nhiêu?" -> "Giá MacBook Pro 14 inch bao nhiêu?" (nếu context đang nói về MacBook)
            - "Có màu nào khác không?" -> "iPhone 15 Pro Max có những màu nào?"
            """),
            ("user", f"""Context hội thoại gần đây:
            {recent_context}
            
            Câu hỏi hiện tại: {user_question}
            
            Câu hỏi tìm kiếm tối ưu:""")
        ])
        
        chain = expansion_prompt | self.llm | StrOutputParser()
        expanded_query = chain.invoke({})
        
        langfuse_context.update_current_observation(
            input=user_question,
            output=expanded_query,
            metadata={"session_id": self.session_id}
        )
        
        return expanded_query.strip()
    
    @observe(name="document_retrieval")
    def retrieve_documents(self, query: str) -> List[Document]:
        """Truy xuất tài liệu liên quan"""
        docs = self.retriever.invoke(query)
        
        langfuse_context.update_current_observation(
            input=query,
            output=[{"content": doc.page_content[:200], "metadata": doc.metadata} for doc in docs],
            metadata={
                "session_id": self.session_id,
                "num_documents": len(docs)
            }
        )
        
        return docs
    
    @observe(name="relevance_check")
    def check_relevance(self, query: str, documents: List[Document]) -> Tuple[List[Document], float]:
        """Kiểm tra độ liên quan của tài liệu với câu hỏi"""
        
        if not documents:
            return [], 0.0
        
        # Tạo prompt để đánh giá độ liên quan
        relevance_prompt = ChatPromptTemplate.from_messages([
            ("system", """Đánh giá độ liên quan của các tài liệu với câu hỏi.
            Trả về điểm từ 0-10 cho mỗi tài liệu (10 = rất liên quan, 0 = không liên quan).
            Chỉ trả về JSON format: {"scores": [score1, score2, ...], "avg_score": average}"""),
            ("user", f"""Câu hỏi: {query}
            
            Tài liệu:
            {chr(10).join([f"Tài liệu {i+1}: {doc.page_content[:300]}" for i, doc in enumerate(documents)])}
            """)
        ])
        
        chain = relevance_prompt | self.llm | StrOutputParser()
        result = chain.invoke({})
        
        try:
            scores_data = json.loads(result)
            scores = scores_data.get("scores", [5.0] * len(documents))
            avg_score = scores_data.get("avg_score", 5.0)
            
            # Lọc tài liệu có điểm >= 6
            relevant_docs = [doc for i, doc in enumerate(documents) if scores[i] >= 6.0]
            
        except:
            # Fallback: giữ nguyên tài liệu
            relevant_docs = documents
            avg_score = 7.0
        
        langfuse_context.update_current_observation(
            input={"query": query, "num_documents": len(documents)},
            output={"relevant_documents": len(relevant_docs), "avg_relevance_score": avg_score},
            metadata={"session_id": self.session_id}
        )
        
        return relevant_docs, avg_score
    
    @observe(name="generate_response")
    def generate_response(self, user_question: str, context_docs: List[Document]) -> str:
        """Tạo câu trả lời dựa trên context"""
        
        # Chuẩn bị context
        context = "\n\n".join([doc.page_content for doc in context_docs])
        
        # Chuẩn bị lịch sử hội thoại
        messages = [SystemMessage(content=self.system_prompt)]
        
        # Thêm lịch sử hội thoại (giới hạn 6 tin nhắn gần nhất)
        recent_history = self.conversation_history[-6:] if len(self.conversation_history) > 0 else []
        for msg in recent_history:
            if msg.role == "user":
                messages.append(HumanMessage(content=msg.content))
            else:
                messages.append(AIMessage(content=msg.content))
        
        # Tạo prompt cho câu hỏi hiện tại
        current_prompt = f"""Thông tin sản phẩm liên quan:
{context}

Câu hỏi của khách hàng: {user_question}

Hãy trả lời câu hỏi dựa trên thông tin được cung cấp ở trên."""
        
        messages.append(HumanMessage(content=current_prompt))
        
        # Tạo response
        response = self.llm.invoke(messages)
        
        langfuse_context.update_current_observation(
            input={"question": user_question, "context_length": len(context)},
            output=response.content,
            metadata={
                "session_id": self.session_id,
                "num_context_docs": len(context_docs)
            }
        )
        
        return response.content
    
    @observe(name="chatbot_conversation")
    def chat(self, user_message: str, intent: str = "general") -> Dict:
        """Xử lý một lượt hội thoại"""
        start_time = time.time()
        
        try:
            # 1. Mở rộng câu hỏi
            expanded_query = self.expand_query(user_message)
            
            # 2. Truy xuất tài liệu
            retrieved_docs = self.retrieve_documents(expanded_query)
            
            # 3. Kiểm tra độ liên quan
            relevant_docs, relevance_score = self.check_relevance(user_message, retrieved_docs)
            
            # 4. Tạo câu trả lời
            if relevant_docs and relevance_score >= 5.0:
                response = self.generate_response(user_message, relevant_docs)
            else:
                response = "Xin lỗi, tôi không tìm thấy thông tin phù hợp để trả lời câu hỏi của bạn. Bạn có thể hỏi về các sản phẩm MacBook, iPhone, AirPods hoặc chính sách bảo hành, vận chuyển không?"
            
            # 5. Lưu vào lịch sử hội thoại
            user_msg = ChatMessage("user", user_message, datetime.now())
            assistant_msg = ChatMessage("assistant", response, datetime.now())
            
            self.conversation_history.append(user_msg)
            self.conversation_history.append(assistant_msg)
            
            # 6. Tính toán metrics
            latency = time.time() - start_time
            
            result = {
                "response": response,
                "metadata": {
                    "session_id": self.session_id,
                    "latency": latency,
                    "relevance_score": relevance_score,
                    "num_relevant_docs": len(relevant_docs),
                    "expanded_query": expanded_query,
                    "intent": intent
                }
            }
            
            # Update LangFuse observation
            langfuse_context.update_current_observation(
                input=user_message,
                output=response,
                metadata=result["metadata"],
                tags=["chatbot", "rag", intent, f"session:{self.session_id}"]
            )
            
            return result
            
        except Exception as e:
            error_response = f"Xin lỗi, đã có lỗi xảy ra: {str(e)}"
            
            langfuse_context.update_current_observation(
                input=user_message,
                output=error_response,
                level="ERROR",
                metadata={"error": str(e), "session_id": self.session_id}
            )
            
            return {
                "response": error_response,
                "metadata": {"error": True, "session_id": self.session_id}
            }
    
    def get_conversation_summary(self) -> Dict:
        """Lấy tóm tắt cuộc hội thoại"""
        return {
            "session_id": self.session_id,
            "total_messages": len(self.conversation_history),
            "start_time": self.conversation_history[0].timestamp if self.conversation_history else None,
            "last_message_time": self.conversation_history[-1].timestamp if self.conversation_history else None
        }

print("🤖 RAGChatbot class đã được định nghĩa!")

## Demo Chatbot và LangFuse Tracking

In [ ]:
# Khởi tạo chatbot
chatbot = RAGChatbot(retriever, llm, langfuse)

print(f"🎯 Chatbot đã sẵn sàng! Session ID: {chatbot.session_id}")
print("\n" + "="*50)
print("DEMO CUỘC HỘI THOẠI")
print("="*50)

In [ ]:
# Demo conversation 1: Hỏi về sản phẩm
print("\n👤 User: Tôi muốn mua laptop để làm việc với thiết kế, bạn có thể giới thiệu không?")

result1 = chatbot.chat(
    "Tôi muốn mua laptop để làm việc với thiết kế, bạn có thể giới thiệu không?",
    intent="product_inquiry"
)

print(f"🤖 Assistant: {result1['response']}")
print(f"\n📊 Metadata: Latency: {result1['metadata']['latency']:.2f}s, Relevance: {result1['metadata']['relevance_score']:.1f}/10")

In [ ]:
# Demo conversation 2: Follow-up question
print("\n👤 User: Giá bao nhiêu và có trả góp không?")

result2 = chatbot.chat(
    "Giá bao nhiêu và có trả góp không?",
    intent="pricing_inquiry"
)

print(f"🤖 Assistant: {result2['response']}")
print(f"\n📊 Metadata: Latency: {result2['metadata']['latency']:.2f}s, Relevance: {result2['metadata']['relevance_score']:.1f}/10")

In [ ]:
# Demo conversation 3: Hỏi về smartphone
print("\n👤 User: Còn điện thoại thì sao? iPhone mới nhất có gì đặc biệt?")

result3 = chatbot.chat(
    "Còn điện thoại thì sao? iPhone mới nhất có gì đặc biệt?",
    intent="product_inquiry"
)

print(f"🤖 Assistant: {result3['response']}")
print(f"\n📊 Metadata: Latency: {result3['metadata']['latency']:.2f}s, Relevance: {result3['metadata']['relevance_score']:.1f}/10")

In [ ]:
# Demo conversation 4: Hỏi về chính sách
print("\n👤 User: Nếu tôi mua cả laptop và điện thoại thì có miễn phí ship không?")

result4 = chatbot.chat(
    "Nếu tôi mua cả laptop và điện thoại thì có miễn phí ship không?",
    intent="policy_inquiry"
)

print(f"🤖 Assistant: {result4['response']}")
print(f"\n📊 Metadata: Latency: {result4['metadata']['latency']:.2f}s, Relevance: {result4['metadata']['relevance_score']:.1f}/10")

In [ ]:
# Xem tóm tắt cuộc hội thoại
summary = chatbot.get_conversation_summary()
print("\n" + "="*50)
print("TÓM TẮT CUỘC HỘI THOẠI")
print("="*50)
print(f"Session ID: {summary['session_id']}")
print(f"Tổng số tin nhắn: {summary['total_messages']}")
print(f"Thời gian bắt đầu: {summary['start_time']}")
print(f"Tin nhắn cuối: {summary['last_message_time']}")

## Đánh giá Tự động và Thu thập Feedback

In [ ]:
class ChatbotEvaluator:
    """Lớp đánh giá chất lượng chatbot"""
    
    def __init__(self, llm, langfuse_client):
        self.llm = llm
        self.langfuse = langfuse_client
    
    @observe(name="hallucination_check")
    def check_hallucination(self, question: str, answer: str, context: str) -> Dict:
        """Kiểm tra hallucination trong câu trả lời"""
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """Bạn là chuyên gia đánh giá độ chính xác của câu trả lời AI.
            Nhiệm vụ: Kiểm tra xem câu trả lời có bịa đặt thông tin không có trong context không.
            
            Trả về JSON:
            {
                "has_hallucination": true/false,
                "confidence": 0.0-1.0,
                "explanation": "giải thích ngắn gọn"
            }"""),
            ("user", f"""Context: {context}
            
            Câu hỏi: {question}
            Câu trả lời: {answer}
            
            Đánh giá:""")
        ])
        
        chain = prompt | self.llm | StrOutputParser()
        result = chain.invoke({})
        
        try:
            evaluation = json.loads(result)
        except:
            evaluation = {
                "has_hallucination": False,
                "confidence": 0.5,
                "explanation": "Không thể phân tích"
            }
        
        langfuse_context.update_current_observation(
            input={"question": question, "answer": answer[:200]},
            output=evaluation,
            metadata={"evaluation_type": "hallucination"}
        )
        
        return evaluation
    
    @observe(name="answer_quality_check")
    def evaluate_answer_quality(self, question: str, answer: str) -> Dict:
        """Đánh giá chất lượng tổng thể của câu trả lời"""
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """Đánh giá chất lượng câu trả lời theo các tiêu chí:
            - Helpfulness (1-5): Câu trả lời có hữu ích không?
            - Clarity (1-5): Câu trả lời có rõ ràng, dễ hiểu không?
            - Completeness (1-5): Câu trả lời có đầy đủ thông tin không?
            - Professional (1-5): Câu trả lời có chuyên nghiệp không?
            
            Trả về JSON:
            {
                "helpfulness": 1-5,
                "clarity": 1-5,
                "completeness": 1-5,
                "professional": 1-5,
                "overall_score": trung_bình,
                "feedback": "nhận xét ngắn"
            }"""),
            ("user", f"""Câu hỏi: {question}
            Câu trả lời: {answer}
            
            Đánh giá:""")
        ])
        
        chain = prompt | self.llm | StrOutputParser()
        result = chain.invoke({})
        
        try:
            evaluation = json.loads(result)
            if "overall_score" not in evaluation:
                scores = [evaluation.get(k, 3) for k in ["helpfulness", "clarity", "completeness", "professional"]]
                evaluation["overall_score"] = sum(scores) / len(scores)
        except:
            evaluation = {
                "helpfulness": 3,
                "clarity": 3,
                "completeness": 3,
                "professional": 3,
                "overall_score": 3.0,
                "feedback": "Không thể đánh giá"
            }
        
        langfuse_context.update_current_observation(
            input={"question": question, "answer": answer[:200]},
            output=evaluation,
            metadata={"evaluation_type": "quality"}
        )
        
        return evaluation
    
    def submit_user_feedback(self, session_id: str, message_id: str, 
                           feedback_type: str, rating: int, comment: str = ""):
        """Thu thập feedback từ người dùng"""
        
        # Tạo score trong LangFuse
        self.langfuse.score(
            trace_id=session_id,  # hoặc observation_id nếu có
            name=f"user_{feedback_type}",
            value=rating,
            comment=comment,
            data_type="NUMERIC"
        )
        
        print(f"✅ Đã ghi nhận feedback: {feedback_type} = {rating}/5")
        if comment:
            print(f"💬 Bình luận: {comment}")

# Khởi tạo evaluator
evaluator = ChatbotEvaluator(llm, langfuse)

print("📋 ChatbotEvaluator đã sẵn sàng!")

In [ ]:
# Demo đánh giá tự động cho cuộc hội thoại vừa rồi
print("\n" + "="*50)
print("ĐÁNH GIÁ TỰ ĐỘNG")
print("="*50)

# Lấy câu hỏi và câu trả lời cuối cùng
last_question = "Nếu tôi mua cả laptop và điện thoại thì có miễn phí ship không?"
last_answer = result4['response']
context = "Chính sách vận chuyển: Miễn phí giao hàng toàn quốc cho đơn hàng từ 10 triệu."

# Kiểm tra hallucination
print("\n🔍 Kiểm tra Hallucination...")
hallucination_check = evaluator.check_hallucination(last_question, last_answer, context)
print(f"Kết quả: {hallucination_check}")

# Đánh giá chất lượng
print("\n⭐ Đánh giá Chất lượng...")
quality_check = evaluator.evaluate_answer_quality(last_question, last_answer)
print(f"Kết quả: {quality_check}")

In [ ]:
# Demo thu thập user feedback
print("\n" + "="*50)
print("THU THẬP USER FEEDBACK")
print("="*50)

# Giả lập feedback từ người dùng
print("\n👤 User feedback cho cuộc hội thoại:")

# Feedback tích cực
evaluator.submit_user_feedback(
    session_id=chatbot.session_id,
    message_id="msg_1",
    feedback_type="helpfulness",
    rating=5,
    comment="Chatbot rất hữu ích, trả lời đúng trọng tâm"
)

evaluator.submit_user_feedback(
    session_id=chatbot.session_id,
    message_id="msg_2",
    feedback_type="satisfaction",
    rating=4,
    comment="Tốt, nhưng có thể cần thêm thông tin về màu sắc sản phẩm"
)

# Feedback tiêu cực
evaluator.submit_user_feedback(
    session_id=chatbot.session_id,
    message_id="msg_3",
    feedback_type="response_time",
    rating=3,
    comment="Phản hồi hơi chậm"
)

## Phân tích Metrics và Tối ưu hóa

In [ ]:
def analyze_chatbot_performance(session_id: str, langfuse_client):
    """Phân tích hiệu suất chatbot từ LangFuse"""
    
    print(f"\n📊 PHÂN TÍCH HIỆU SUẤT CHATBOT")
    print(f"Session ID: {session_id}")
    print("="*50)
    
    # Lấy dữ liệu từ LangFuse (thực tế cần sử dụng API)
    # Ở đây chúng ta sẽ mô phỏng dữ liệu
    
    mock_data = {
        "total_conversations": 4,
        "avg_latency": 2.3,
        "avg_relevance_score": 8.2,
        "user_satisfaction": 4.2,
        "hallucination_rate": 0.05,
        "intent_distribution": {
            "product_inquiry": 50,
            "pricing_inquiry": 25,
            "policy_inquiry": 25
        },
        "token_usage": {
            "total_input_tokens": 1250,
            "total_output_tokens": 890,
            "total_cost_usd": 0.045
        }
    }
    
    print(f"\n🎯 KPI Chính:")
    print(f"  • Tổng số cuộc hội thoại: {mock_data['total_conversations']}")
    print(f"  • Độ trễ trung bình: {mock_data['avg_latency']:.1f}s")
    print(f"  • Điểm liên quan trung bình: {mock_data['avg_relevance_score']:.1f}/10")
    print(f"  • Độ hài lòng người dùng: {mock_data['user_satisfaction']:.1f}/5")
    print(f"  • Tỷ lệ hallucination: {mock_data['hallucination_rate']*100:.1f}%")
    
    print(f"\n🎭 Phân bố Intent:")
    for intent, percentage in mock_data['intent_distribution'].items():
        print(f"  • {intent}: {percentage}%")
    
    print(f"\n💰 Chi phí Token:")
    print(f"  • Input tokens: {mock_data['token_usage']['total_input_tokens']:,}")
    print(f"  • Output tokens: {mock_data['token_usage']['total_output_tokens']:,}")
    print(f"  • Tổng chi phí: ${mock_data['token_usage']['total_cost_usd']:.3f}")
    
    # Đề xuất tối ưu hóa
    print(f"\n🚀 Đề xuất Tối ưu hóa:")
    
    if mock_data['avg_latency'] > 3.0:
        print("  ⚠️  Độ trễ cao - Cần tối ưu hóa retrieval hoặc caching")
    else:
        print("  ✅ Độ trễ trong mức chấp nhận được")
    
    if mock_data['avg_relevance_score'] < 7.0:
        print("  ⚠️  Điểm liên quan thấp - Cần cải thiện embeddings hoặc chunking")
    else:
        print("  ✅ Độ liên quan tài liệu tốt")
    
    if mock_data['user_satisfaction'] < 4.0:
        print("  ⚠️  Độ hài lòng chưa cao - Cần cải thiện prompt và training data")
    else:
        print("  ✅ Người dùng hài lòng với chatbot")
    
    if mock_data['hallucination_rate'] > 0.1:
        print("  ⚠️  Tỷ lệ hallucination cao - Cần tăng cường fact-checking")
    else:
        print("  ✅ Tỷ lệ hallucination trong mức an toàn")
    
    return mock_data

# Phân tích hiệu suất
performance_data = analyze_chatbot_performance(chatbot.session_id, langfuse)

## Cải tiến Prompt và Fine-tuning

In [ ]:
# Ví dụ về prompt optimization dựa trên feedback
def optimize_system_prompt(performance_metrics: Dict) -> str:
    """Tối ưu hóa system prompt dựa trên performance metrics"""
    
    base_prompt = """Bạn là trợ lý AI chuyên nghiệp hỗ trợ khách hàng về các sản phẩm Apple."""
    
    improvements = []
    
    if performance_metrics['hallucination_rate'] > 0.05:
        improvements.append(
            "QUAN TRỌNG: Chỉ sử dụng thông tin có trong tài liệu được cung cấp. "
            "Tuyệt đối không bịa đặt thông tin."
        )
    
    if performance_metrics['user_satisfaction'] < 4.0:
        improvements.append(
            "Luôn:"
            "- Cung cấp thông tin cụ thể và hữu ích\n"
            "- Sử dụng ngôn ngữ thân thiện, dễ hiểu\n"
            "- Đề xuất sản phẩm phù hợp với nhu cầu khách hàng"
        )
    
    if performance_metrics['avg_relevance_score'] < 7.0:
        improvements.append(
            "Khi trả lời:"
            "- Tập trung vào thông tin trực tiếp liên quan đến câu hỏi\n"
            "- Ưu tiên thông tin quan trọng nhất trước\n"
            "- Nếu câu hỏi không rõ ràng, hỏi lại để làm rõ"
        )
    
    optimized_prompt = base_prompt
    if improvements:
        optimized_prompt += "\n\n" + "\n\n".join(improvements)
    
    return optimized_prompt

# Tạo prompt được tối ưu hóa
optimized_prompt = optimize_system_prompt(performance_data)

print("\n" + "="*50)
print("PROMPT OPTIMIZATION")
print("="*50)
print("\n🔧 System Prompt đã được tối ưu hóa:")
print("\n" + optimized_prompt)

# Cập nhật prompt cho chatbot
chatbot.system_prompt = optimized_prompt
print("\n✅ Đã cập nhật system prompt cho chatbot!")

## Test với Prompt Mới

In [ ]:
# Test chatbot với prompt đã được tối ưu hóa
print("\n" + "="*50)
print("TEST VỚI PROMPT ĐÃ TỐI ƯU HÓA")
print("="*50)

print("\n👤 User: Tôi có budget 30 triệu, muốn mua điện thoại chụp ảnh đẹp. Có gợi ý gì không?")

result_optimized = chatbot.chat(
    "Tôi có budget 30 triệu, muốn mua điện thoại chụp ảnh đẹp. Có gợi ý gì không?",
    intent="product_recommendation"
)

print(f"🤖 Assistant (Optimized): {result_optimized['response']}")
print(f"\n📊 Metadata: Latency: {result_optimized['metadata']['latency']:.2f}s, Relevance: {result_optimized['metadata']['relevance_score']:.1f}/10")

# So sánh performance
print("\n📈 So sánh Performance:")
print(f"Độ trễ: {result_optimized['metadata']['latency']:.2f}s")
print(f"Độ liên quan: {result_optimized['metadata']['relevance_score']:.1f}/10")
print(f"Expanded query: {result_optimized['metadata']['expanded_query']}")

## Giải thích & Phân tích

### 🎯 Lợi ích của LangFuse trong RAG Chatbot:

1. **End-to-End Observability**:
   - Theo dõi toàn bộ pipeline từ query expansion → retrieval → generation
   - Đo lường latency và token usage của từng bước
   - Phát hiện bottlenecks và tối ưu hóa hiệu suất

2. **Quality Assurance**:
   - Đánh giá tự động hallucination và chất lượng câu trả lời
   - Thu thập feedback người dùng thực tế
   - Tracking metrics quan trọng: relevance, satisfaction, accuracy

3. **Continuous Improvement**:
   - Phân tích pattern từ dữ liệu thực tế
   - A/B testing các version prompt khác nhau
   - Tối ưu hóa dựa trên user behavior và feedback

4. **Production Monitoring**:
   - Real-time alerts khi có issues
   - Cost tracking và budget management
   - Performance benchmarking theo thời gian

### 🔍 Insights từ Demo:

- **Multi-turn Conversation**: LangFuse tracking giúp hiểu context flow qua nhiều lượt hội thoại
- **Intent Classification**: Tags giúp phân tích user behavior patterns
- **Quality Control**: Automated evaluation giúp detect issues sớm
- **User Experience**: Feedback collection cung cấp ground truth cho improvement

### 🚀 Production Best Practices:

1. **Monitoring Strategy**:
   - Set up alerts cho latency > threshold
   - Monitor hallucination rate daily
   - Track user satisfaction trends

2. **Data Collection**:
   - Collect diverse training examples
   - Annotate good/bad responses
   - Regular evaluation dataset updates

3. **Iterative Improvement**:
   - Weekly prompt optimization
   - Monthly model evaluation
   - Quarterly architecture review

## Tài liệu Tham khảo

### 📚 LangFuse Documentation:
- [Tracing](https://langfuse.com/docs/tracing)
- [Python SDK](https://langfuse.com/docs/sdk/python)
- [Evaluation](https://langfuse.com/docs/scores/overview)
- [Prompt Management](https://langfuse.com/docs/prompts/get-started)

### 🔗 LangChain Integration:
- [LangChain Callback](https://langfuse.com/docs/integrations/langchain/tracing)
- [RAG with LangFuse](https://langfuse.com/docs/cookbook/integration_langchain_rag)

### 📖 RAG Best Practices:
- [Advanced RAG Techniques](https://langfuse.com/blog/rag-evaluation)
- [Production RAG Systems](https://langfuse.com/docs/cookbook/rag_evaluation_with_ragas)

### 🎯 Evaluation & Monitoring:
- [Human Feedback](https://langfuse.com/docs/scores/user-feedback)
- [Model-based Evaluation](https://langfuse.com/docs/scores/model-based-evals)
- [Production Analytics](https://langfuse.com/docs/analytics)

## Kết luận & Bước tiếp theo

### ✅ Những gì đã học:
- Thiết kế và triển khai RAG chatbot với multi-turn conversation
- Sử dụng LangFuse để theo dõi và đánh giá hiệu suất
- Triển khai automated evaluation và user feedback collection
- Tối ưu hóa prompt dựa trên dữ liệu thực tế

### 🎯 Bước tiếp theo:

1. **Mở rộng Knowledge Base**:
   - Thêm nhiều loại tài liệu (video, audio, structured data)
   - Implement semantic chunking strategies
   - Multi-modal RAG với images

2. **Advanced Features**:
   - Tool calling integration (inventory check, order placement)
   - Multi-language support
   - Personalization based on user history

3. **Production Deployment**:
   - Containerize với Docker
   - Setup CI/CD pipeline
   - Load balancing và scaling

4. **Advanced Analytics**:
   - Customer journey analysis
   - Conversion rate optimization
   - A/B testing framework

### 💡 Key Takeaways:
- **Observability is crucial** cho production RAG systems
- **User feedback** là ground truth quan trọng nhất
- **Iterative improvement** dựa trên data là chìa khóa thành công
- **LangFuse provides comprehensive tooling** cho toàn bộ ML lifecycle

Chúc mừng! Bạn đã thành thạo việc xây dựng và tối ưu hóa RAG chatbot với LangFuse! 🎉